In [ ]:
!pip install  cassandra-driver
!pip install  cassio>=0.1.1
!pip install  gradientai --upgrade
!pip install  llama-index
!pip install  pypdf
!pip install  tiktoken==0.4.0

In [ ]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = userdata.get('GRADIENT_ACCESS_TOKEN')
os.environ['GRADIENT_WORKSPACE_ID'] = userdata.get('GRADIENT_WORKSPACE_ID')

In [ ]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

In [ ]:
import cassandra as cd
print(cd.__version__)

3.29.0


In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-temp-db.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("temp_db-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137344063195792) 0920c9fa-a568-426c-9446-497f3efc5b88-us-east1.db.astra.datastax.com:29042:19107618-1dd0-4139-add7-247eb537830e> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-c44bc828c0c8


In [ ]:
llm = GradientBaseModelLLM(base_model_slug='llama2-7b-chat',max_tokens=400)

In [ ]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug = "bge-large",
)

In [ ]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size = 256,
)
set_global_service_context(service_context)

In [ ]:
!pip install docx2txt

In [ ]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)}  documents(s).")

Loaded 54  documents(s).


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context = service_context)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What is Quantum Computing?")
print(response)

 Quantum Computing is a rapidly emerging technology used for solving problems that are too complex for classical computers to solve. These machines are very different from normal home computers and classical ones. They use the laws of quantum mechanics in Information technology for their computations. Quantum computers show promise in solving optimization problems more efficiently in many fields such as cryptography, financial modeling, and so on. They have more computational power so it only makes sense to assume that quantum computers have better outcomes than the already existing Machine Learning algorithms. It enhances Machine Learning tasks such as complex optimization problems encountered in deep neural networks or improving feature selection for large datasets.
